In [2]:
!pip install fosforml
!pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [ ]:
df = 'ATTRITION_PREDICTIVE_DATA'

In [ ]:
sf_df = my_session.sql("select * from {}".format(df))

In [ ]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [ ]:
print(pandas_df.isnull().sum())

In [ ]:
Original_df = pandas_df.dropna()

In [ ]:
print(Original_df.isnull().sum())

In [ ]:
df = Original_df[["DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP","JOB_ENDDATE","CHURN"]]

In [ ]:
df

In [ ]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])

In [ ]:
df.set_index('JOB_ENDDATE', inplace = True)

In [ ]:
New_df = df.groupby(["DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP"]).resample('M').agg({'CHURN': 'sum'}).reset_index()

In [ ]:
New_df

In [ ]:
df1 = New_df[New_df['CHURN'] > 0 ]

In [ ]:
df1

In [ ]:
# Split data into training and testing sets
train = df1[:'2024-08']
test = df1['2024-09':]

In [ ]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])
monthly_data = df.resample('M', on='JOB_ENDDATE').sum()

In [ ]:
monthly_data